In [ ]:
key='api_key'

In [2]:
!pip install -q -U google-generativeai

In [4]:
# Gemini lib (REST-wrapper)
import google.generativeai as genai
import time
import json

# Kaggle lib for secrets

import pandas as pd
import numpy as np

api_key = key
genai.configure(api_key=api_key)

In [21]:
generation_config = {
    "temperature": 0.1,
    "top_p": 0.1,
    "top_k": 1,
    "max_output_tokens": 5000,  # Adjust as needed
}
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
]
MODEL_NAME = "gemini-2.0-flash"
model = genai.GenerativeModel(MODEL_NAME,
                            generation_config=generation_config,
                            safety_settings=safety_settings)

In [8]:
id=1
df=pd.read_csv(f'./fdr/{id}/fdr.csv')
context = open(f'./fdr/{id}/context.txt','r').readlines()
columns = df.columns
value_units = df.iloc[0, :].values
value_type_legend = df.iloc[1, :].values
params=df.columns
all_results =[]
data=df.loc[2:,:]
data.shape

C:\Users\serve\AppData\Local\Temp\ipykernel_8532\3516806462.py:2: DtypeWarning: Columns (1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(f'./fdr/{id}/fdr.csv')


(238742, 23)

In [37]:
all_results[-1]

'```'

In [45]:
generation_config = {
    "temperature": 0.0,
    "top_p": 0.1,
    "top_k": 1,
    "max_output_tokens": 5000,  # Adjust as needed
}
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_LOW_AND_ABOVE",
    },
]
MODEL_NAME = "gemini-1.5-pro-latest"
model = genai.GenerativeModel(MODEL_NAME,
                            generation_config=generation_config,
                            safety_settings=safety_settings)

In [50]:
context = "Aircraft Type:Airbus A320, midnight was at: 01/15/09 [mm/dd/yy] 00:00:00 [HH:MM:ss]"
columns = df.columns
value_units = df.iloc[0, :].values
value_type_legend = df.iloc[1, :].values
all_results =[]

In [47]:
transcript=pd.read_csv('output.csv')

In [60]:

prompt_text = f"""
    I have flight data records for {context}.
    Flight data parameters: {list(columns[1:])},
    where the units in which each of the parameters where applicable are : {list(value_units)},
    and the types of values for each parameter are : {list(value_type_legend)}.
    
    I have the transcript of the conversation between the Air traffic control operator and the pilot for the corresponding period.
    Conversation data: 
    {transcript}

    analyze the conversation data and based on that and give me the following are three seperate groups of outputs:
    1.give me the list of flight data parameters from the parameters I have given as a list to focus on for further analysis (the parameters should be from the list of flight data parameters provided for this question)
    2.what is the possible issue detected from the conversation?
    3.what aircraft components would be affected based on the analyzed issue from the conversation?
    4.what are the suggestions given by the air traffic controller?
    5.what actions were taken by the pilot?
    6.what are the next steps taken or to be taken by the pilot?
    
    
    the data is accurate and is completely reliable.
    
    I want the response to each of the questions 1 -6 as a json object.
    """
response = model.generate_content(prompt_text)
response.resolve()
if response.parts:
    analysis_results = response.parts[0].text.strip().split('\n')
    all_results.extend(analysis_results)
else:
    all_results.append("No analysis provided.")

In [61]:
all_results

['```json',
 '{',
 '  "1_flight_data_parameters_to_focus": [',
 '    "Eng1 N1 Act", ',
 '    "Eng1 N1 Cmd",',
 '    "Eng1 N2",',
 '    "Eng1 Fuel Flow",',
 '    "Eng1 EGT",',
 '    "Eng1 Fire",',
 '    "Eng2 N1 Act",',
 '    "Eng2 N1 Cmd",',
 '    "Eng2 N2",',
 '    "Eng2 Fuel Flow",',
 '    "Eng2 EGT",',
 '    "Eng2 Fire",',
 '    "Altitude Radio 1",',
 '    "Altitude Radio 2",',
 '    "Airspeed Ind",',
 '    "Pitch",',
 '    "Roll",',
 '    "Heading",',
 '    "Latitude",',
 '    "Longitude",',
 '    "Flap Lever Pos",',
 '    "Flap Pos",',
 '    "Slat Pos",',
 '    "Spoiler 1 Status",',
 '    "Spoiler 2 Status",',
 '    "Spoiler 3 Status",',
 '    "Spoiler 4 Status",',
 '    "Spoiler 5 Status",',
 '    "Spd Brake Cmnd",',
 '    "Gear Select Down",',
 '    "Gear Down Locked",',
 '    "Ground Speed",',
 '    "Event" ',
 '  ],',
 '  "2_possible_issue": {',
 '    "issue": "Dual engine thrust loss/failure."',
 '  },',
 '  "3_affected_aircraft_components": {',
 '    "components": [',
 '    

In [62]:
transcript_result=all_results.copy()

In [63]:
params=["Eng1 N1 Act","Eng1 N1 Cmd","Eng1 N2","Eng1 Fuel Flow","Eng1 EGT","Eng2 N1 Act","Eng2 N1 Cmd","Eng2 N2","Eng2 Fuel Flow","Eng2 EGT","Altitude Radio 1","Altitude Radio 2","Airspeed Ind","Pitch","Roll ","Heading",
 "Flap Lever Pos","Flap Pos","Slat Pos","Spoiler 1 Status","Spoiler 2 Status","Spoiler 3 Status","Spoiler 4 Status","Spoiler 5 Status","Event","Latitude","Longitude"]

In [64]:
new_df=df.iloc[2:].copy()

In [65]:
def conv_est(x):
    return int(float(x))
new_df['EST_full_second']=new_df['EST'].apply(conv_est)
fin_df=new_df.loc[new_df[params].dropna(how='all').index][[new_df.columns[0]]+[new_df.columns[-1]]+params]
fin_df1=fin_df.drop(['EST_full_second'],axis=1).copy()
mast_list=[]
for i, j in fin_df1.loc[fin_df1.index].iterrows():
    mast_list+=[j.dropna().to_dict()]

In [66]:
all_results=[]
for i in range(0,fin_df1.shape[0], fin_df1.shape[0]//20):
    part1= f"""
        I have flight data records for {context}.
        The detected issue with the flight is : Dual engine thrust loss/failure
        Flight data parameters: {list(columns)},
        where the units in which each of the parameters where applicable are : {list(value_units)},
        and the types of values for each parameter are : {list(value_type_legend)}.
    
        I have a list of flight data information which is stored as a dictionary whose key consists of 'EST' and some other keys from flight the list of parameters provided in Flightdata parameters and the values are the corresponding values for each key.
        flight data: 
        {mast_list[i:min(i + fin_df1.shape[0]//20, fin_df1.shape[0])]}
    
        Analyze the flight data and give me the values from the EST column for which anamolies are detected in the flight parametes or where the flight data parametes are suspicious or abnormal.
        Furthermore, include the name of the parameter or parameters  that shopws the most risk and their corresponding values. 
             
        
        The data is accurate and is completely reliable.
        
        I want the response as a list of json objects for each risky EST column value with the following keys:
        1. 'EST'
        2. 'risk_parameters'
        3. 'risk_parameter_values'
    
        the values from 1-3 should be the only output
        """
    part2="""
        [{'EST': , 'risk_parameters': , 'risk_parameter_values': }]
        I want the output in exactly this format and nothign else
        """
    prompt_text =part1
    
        
    response = model.generate_content(prompt_text)
    response.resolve()
    if response.parts:
        analysis_results = response.parts[0].text.strip().split('\n')
        all_results.extend(analysis_results)
    else:
        all_results.append("No analysis provided.")
    time.sleep(65)

In [67]:
flight_data_analysis=all_results.copy()

In [68]:
flight_data_analysis

['```json',
 '[',
 '  {',
 '    "EST": "54404.5919375",',
 '    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],',
 '    "risk_parameter_values": ["1.500", "0"]',
 '  },',
 '  {',
 '    "EST": "54404.59975",',
 '    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],',
 '    "risk_parameter_values": ["1.500", "0"]',
 '  },',
 '    {',
 '    "EST": "54405.5919375",',
 '    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],',
 '    "risk_parameter_values": ["3.875", "0"]',
 '  },',
 '  {',
 '    "EST": "54405.59975",',
 '    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],',
 '    "risk_parameter_values": ["3.875", "0"]',
 '  },',
 '  {',
 '    "EST": "54406.5919375",',
 '    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],',
 '    "risk_parameter_values": ["6.125", "0"]',
 '  },',
 '  {',
 '    "EST": "54406.59975",',
 '    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],',
 '    "risk_parameter_values": ["6.125", "0"]',
 '  },',
 '  {',
 '    "EST": "54407.5919375",',
 '    "risk_para

In [111]:
#from flight_data_analysis
json_data1 = """[
  {
    "EST": "54404.5919375",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["1.500", "0"]
  },
  {
    "EST": "54404.59975",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["1.500", "0"]
  },
  {
    "EST": "54405.5919375",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["3.875", "0"]
  },
  {
    "EST": "54405.59975",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["3.875", "0"]
  },
  {
    "EST": "54406.5919375",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["6.125", "0"]
  },
  {
    "EST": "54406.59975",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["6.125", "0"]
  },
  {
    "EST": "54407.5919375",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["10.250", "0"]
  },
  {
    "EST": "54407.59975",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["10.250", "0"]
  },
  {
    "EST": "54424.5919375",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["59.500", "688"]
  },
  {
    "EST": "54424.59975",
    "risk_parameters": ["Eng1 N2", "Eng1 Fuel Flow"],
    "risk_parameter_values": ["59.500", "688"]
  }
]"""

json_data2 = """[
  {
    "EST": 54462.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54464.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54467.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54468.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54469.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54470.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54471.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54472.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54473.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54474.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54475.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54476.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54477.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54478.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54479.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54480.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54481.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54482.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54483.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54484.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54485.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54486.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54487.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54488.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54489.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54490.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54491.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54492.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54493.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54494.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54495.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54496.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54497.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54498.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54499.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54500.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54501.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54502.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54503.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54504.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54505.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54506.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54507.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54508.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54509.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54510.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54511.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54512.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54513.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54514.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54515.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54516.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54517.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54518.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54519.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54520.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54521.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54522.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54523.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54524.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54525.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54526.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54527.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54528.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54529.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54530.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54531.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54532.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  },
  {
    "EST": 54533.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [0.0]
  },
  {
    "EST": 54534.021625,
    "risk_parameters": ["Airspeed Ind"],
    "risk_parameter_values": [511.875]
  }
]"""


def process_json_data(json_str):
    data = json.loads(json_str)
    rows = []
    for entry in data:
        row = {}
        est_val = entry.get("EST")
        if isinstance(est_val, str):
            row["EST"] = int(float(est_val))
        else:
            row["EST"] = int(est_val)
        
        for param, value in zip(entry.get("risk_parameters", []), entry.get("risk_parameter_values", [])):
            row[param] = value
        rows.append(row)
    return rows


rows1 = process_json_data(json_data1)
rows2 = process_json_data(json_data2)


all_rows = rows1 + rows2

risk_columns = set()
for row in all_rows:
    risk_columns.update(row.keys())
risk_columns.discard("EST")
risk_columns = sorted(list(risk_columns)) 

final_columns = ["EST"] + risk_columns

js_df = pd.DataFrame(all_rows, columns=final_columns)

csv_filename = "flight_data_converted.csv"
js_df.to_csv(csv_filename, index=False)


In [112]:
js_df

,EST,Airspeed Ind,Eng1 Fuel Flow,Eng1 N2
0,54404,NaN,0,1.500
1,54404,NaN,0,1.500
2,54405,NaN,0,3.875
3,54405,NaN,0,3.875
4,54406,NaN,0,6.125
...,...,...,...,...
75,54530,0.000,NaN,NaN
76,54531,0.000,NaN,NaN
77,54532,511.875,NaN,NaN
78,54533,0.000,NaN,NaN


In [113]:
js_df['Event']=1
js_df

,EST,Airspeed Ind,Eng1 Fuel Flow,Eng1 N2,Event
0,54404,NaN,0,1.500,1
1,54404,NaN,0,1.500,1
2,54405,NaN,0,3.875,1
3,54405,NaN,0,3.875,1
4,54406,NaN,0,6.125,1
...,...,...,...,...,...
75,54530,0.000,NaN,NaN,1
76,54531,0.000,NaN,NaN,1
77,54532,511.875,NaN,NaN,1
78,54533,0.000,NaN,NaN,1


In [114]:
js_df.rename(columns={'EST':'EST_full_second'},inplace=True)

In [115]:
new_df.shape

(146064, 178)

In [116]:
out_df=pd.merge(new_df[['EST','EST_full_second']],js_df,on='EST_full_second',how='outer')
out_df

,EST,EST_full_second,Airspeed Ind,Eng1 Fuel Flow,Eng1 N2,Event
0,54388.8888125,54388,NaN,NaN,NaN,NaN
1,54388.896625,54388,NaN,NaN,NaN,NaN
2,54388.9044375,54388,NaN,NaN,NaN,NaN
3,54388.91225,54388,NaN,NaN,NaN,NaN
4,54388.9200625,54388,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
146560,55843.295063,55843,NaN,NaN,NaN,NaN
146561,55843.302875,55843,NaN,NaN,NaN,NaN
146562,55843.310688,55843,NaN,NaN,NaN,NaN
146563,55843.3185,55843,NaN,NaN,NaN,NaN


In [118]:
out_df['Event']=out_df['Event'].fillna(0)
out_df['Event']=out_df['Event'].astype(int)
out_df['Event'].value_counts(dropna=False)

Event
0    138536
1      8029
Name: count, dtype: int64

In [139]:
def aspd(x):
    if(x<0):
        return 0
    else:
        return 100

In [120]:
def flflw(x):
    if(np.isnan(x)):
        return 0
    else:
        return 100

In [156]:
out_df=out_df1.copy()

In [148]:
events=['Airspeed Ind', 'Eng1 Fuel Flow', 'Eng1 N2']

In [152]:
out_df.columns

Index(['EST', 'EST_full_second', 'Airspeed Ind', 'Eng1 Fuel Flow', 'Eng1 N2',
       'Event'],
      dtype='object')

In [157]:
out_df.fillna(-100000,inplace=True)
out_df[events] = out_df[events].astype(float)

In [158]:
out_df

,EST,EST_full_second,Airspeed Ind,Eng1 Fuel Flow,Eng1 N2,Event
0,54388.8888125,54388,-100000.0,-100000.0,-100000.0,0
1,54388.896625,54388,-100000.0,-100000.0,-100000.0,0
2,54388.9044375,54388,-100000.0,-100000.0,-100000.0,0
3,54388.91225,54388,-100000.0,-100000.0,-100000.0,0
4,54388.9200625,54388,-100000.0,-100000.0,-100000.0,0
...,...,...,...,...,...,...
146560,55843.295063,55843,-100000.0,-100000.0,-100000.0,0
146561,55843.302875,55843,-100000.0,-100000.0,-100000.0,0
146562,55843.310688,55843,-100000.0,-100000.0,-100000.0,0
146563,55843.3185,55843,-100000.0,-100000.0,-100000.0,0


In [159]:
#out_df['Airspeed Ind']=new_df['Airspeed Ind'].apply(aspd)
import numpy as np
c=1
for event in events:
    out_df.loc[out_df[event] > -100000.0, event] = 10**c
    out_df.loc[out_df[event] == -100000.0, event] = 0
    c+=1

In [162]:
out_df['sum']=out_df['Airspeed Ind']+out_df['Eng1 Fuel Flow']+out_df['Eng1 N2']
out_df

,EST,EST_full_second,Airspeed Ind,Eng1 Fuel Flow,Eng1 N2,Event,sum
0,54388.8888125,54388,0.0,0.0,0.0,0,0.0
1,54388.896625,54388,0.0,0.0,0.0,0,0.0
2,54388.9044375,54388,0.0,0.0,0.0,0,0.0
3,54388.91225,54388,0.0,0.0,0.0,0,0.0
4,54388.9200625,54388,0.0,0.0,0.0,0,0.0
...,...,...,...,...,...,...,...
146560,55843.295063,55843,0.0,0.0,0.0,0,0.0
146561,55843.302875,55843,0.0,0.0,0.0,0,0.0
146562,55843.310688,55843,0.0,0.0,0.0,0,0.0
146563,55843.3185,55843,0.0,0.0,0.0,0,0.0


In [ ]:
out_df['Airspeed Ind']+out_df['Eng1 Fuel Flow']+out_df['Eng1 N2']

1.01

In [173]:
def get_issue(x):
    s=[]
    if(x/1000>=1):
        s+=['Eng1 N2']
    elif(x/100>=1):
        s+=['Eng1 Fuel Flow']
    elif(x/10>=1):
        s+=['Airspeed Ind']
    return ', '.join(s)

In [174]:
out_df['issue']=out_df['sum'].apply(get_issue)
out_df[out_df['Event']==1]

,EST,EST_full_second,Airspeed Ind,Eng1 Fuel Flow,Eng1 N2,Event,sum,issue
1524,54404.0138125,54404,0.0,100.0,1000.0,1,1100.0,Eng1 N2
1525,54404.0138125,54404,0.0,100.0,1000.0,1,1100.0,Eng1 N2
1526,54404.021625,54404,0.0,100.0,1000.0,1,1100.0,Eng1 N2
1527,54404.021625,54404,0.0,100.0,1000.0,1,1100.0,Eng1 N2
1528,54404.0294375,54404,0.0,100.0,1000.0,1,1100.0,Eng1 N2
...,...,...,...,...,...,...,...,...
15173,54534.966938,54534,10.0,0.0,0.0,1,10.0,Airspeed Ind
15174,54534.97475,54534,10.0,0.0,0.0,1,10.0,Airspeed Ind
15175,54534.982563,54534,10.0,0.0,0.0,1,10.0,Airspeed Ind
15176,54534.990375,54534,10.0,0.0,0.0,1,10.0,Airspeed Ind


In [175]:
fin_out_df=out_df[['EST','Event','issue']]

In [177]:
fin_out_df.to_csv('output.csv')

In [129]:
max(out_df1['Airspeed Ind'].dropna())

511.875

In [109]:
out_df['Eng1 Fuel Flow']=new_df['Eng1 Fuel Flow'].apply(flflw)
out_df

,EST,EST_full_second,Airspeed Ind,Eng1 Fuel Flow,Eng1 N2,Event
0,54388.8888125,54388,NaN,NaN,NaN,0
1,54388.896625,54388,NaN,NaN,NaN,0
2,54388.9044375,54388,100.0,0.0,NaN,0
3,54388.91225,54388,100.0,0.0,NaN,0
4,54388.9200625,54388,100.0,0.0,NaN,0
...,...,...,...,...,...,...
146560,55843.295063,55843,NaN,NaN,NaN,0
146561,55843.302875,55843,NaN,NaN,NaN,0
146562,55843.310688,55843,NaN,NaN,NaN,0
146563,55843.3185,55843,NaN,NaN,NaN,0
